<a href="https://colab.research.google.com/github/Dinuda/-Shell/blob/main/urls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [7]:
import os
import pandas as pd
import numpy as np
import random
from copy import deepcopy
from itertools import chain

## Utils



In [8]:
import requests

api_base = 'https://en.wikipedia.org/w/api.php'

def get_category_members(title, cmcontinue):
  params = {
      "action": "query",
      "format": "json",
      "list": "categorymembers",
      "cmtitle": title,
      "cmlimit": "500"
  }

  if(cmcontinue):
    params["cmcontinue"] = cmcontinue

  response = requests.get(api_base, params)

  return response.json()

In [14]:
def get_all_category_members(title, depth=1):

  page_urls = []

  # check depth of category
  if depth < 0:
    return page_urls

  category_members = get_category_members(title, cmcontinue=None)

  while True:
    for member in category_members["query"]["categorymembers"]:
      page_title = member["title"].replace(" ", "_")
      page_url = f"https://en.wikipedia.org/wiki/{page_title}"

      # check if each member is a page or has sub categories
      if "Category:" in member["title"]:
        # fetch recursively
        page_urls.extend(get_all_category_members(member["title"], depth=depth))
      else:
        page_urls.append(page_url)

      if 'continue' in category_members:
        cmcontinue = category_members['continue']['cmcontinue']
        print(cmcontinue)
      else:
        break

    return page_urls

In [15]:
url_bank = set()

In [16]:
category2depth = {
    "Category:Concepts_in_physics": 3,
    "Category:Subfields of physics": 2,
    "Category:Physical_systems": 2,
    "Category:Physical modeling": 1,
}

In [17]:
%%time
url_list = []

for cat, depth in category2depth.items():
    print(f"Getting URLs for {cat} with depth {depth}")
    urls = get_all_category_members(cat, depth=depth)
    print(f"# urls extracted = {len(urls)}")
    url_list.append(urls)

Getting URLs for Category:Concepts_in_physics with depth 3
# urls extracted = 1
Getting URLs for Category:Subfields of physics with depth 2
# urls extracted = 1
Getting URLs for Category:Physical_systems with depth 2
# urls extracted = 1
Getting URLs for Category:Physical modeling with depth 1
# urls extracted = 1
CPU times: user 229 ms, sys: 2.81 ms, total: 232 ms
Wall time: 759 ms
